In [1]:
!pwd

/home/joeri/UN-PML-Pilot


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [9]:
#Run this cell for training with all original data

""" X_train=pd.read_csv("./DATA/UCI_HAR_Dataset/train/X_train.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])
y_train=pd.read_csv("./DATA/UCI_HAR_Dataset/train/y_train.txt", delim_whitespace=True, names=["label"])
y_train['label']=y_train['label']-1

X_test=pd.read_csv("./DATA/UCI_HAR_Dataset/test/X_test.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])
y_test=pd.read_csv("./DATA/UCI_HAR_Dataset/test/y_test.txt", delim_whitespace=True, names=["label"])
y_test['label']=y_test['label']-1 """

' X_train=pd.read_csv("./DATA/UCI_HAR_Dataset/train/X_train.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])\ny_train=pd.read_csv("./DATA/UCI_HAR_Dataset/train/y_train.txt", delim_whitespace=True, names=["label"])\ny_train[\'label\']=y_train[\'label\']-1\n\nX_test=pd.read_csv("./DATA/UCI_HAR_Dataset/test/X_test.txt", delim_whitespace=True, names=["F"+str(i) for i in range(1, 562)])\ny_test=pd.read_csv("./DATA/UCI_HAR_Dataset/test/y_test.txt", delim_whitespace=True, names=["label"])\ny_test[\'label\']=y_test[\'label\']-1 '

In [4]:
#Run this cell for training with data subset in format after running splitting script

train_path = "benchmark/benchmark_data/0/train/ALL_train.csv"
test_path = "benchmark/benchmark_data/0/test/ALL_test.csv"

trainingset = pd.read_csv(train_path, delimiter=';')
testset = pd.read_csv(test_path, delimiter=';')

X_train = pd.concat([trainingset[str(i)]
                    for i in range(561)], axis=1)
y_train = trainingset['Y'] - 1

X_test = pd.concat([testset[str(i)] for i in range(561)], axis=1)
y_test = testset['Y'] - 1


In [5]:
trainDF = pd.concat([X_train,y_train],axis=1) 

testDF=pd.concat([X_test, y_test],axis=1)

In [6]:
trainDF.values

array([[ 0.22184689,  0.03410767, -0.12361242, ..., -0.02075593,
        -0.25526977,  3.        ],
       [ 0.23715407,  0.00782512, -0.12283791, ...,  0.00614111,
        -0.21275283,  3.        ],
       [ 0.27394057, -0.01314758, -0.10543517, ..., -0.01704646,
        -0.20844898,  3.        ],
       ...,
       [ 0.20754233, -0.04869768, -0.05346682, ...,  0.14532692,
         0.00870735,  2.        ],
       [ 0.22328281,  0.01391053, -0.21415242, ...,  0.25322243,
         0.01445158,  1.        ],
       [ 0.29966534, -0.05719341, -0.18123302, ...,  0.23860439,
         0.04981914,  1.        ]])

In [7]:
training_data = torch.utils.data.TensorDataset(torch.tensor(X_train.values).float(), torch.as_tensor(y_train.values).squeeze())
test_data=torch.utils.data.TensorDataset(torch.tensor(X_test.values).float(), torch.as_tensor(y_test.values).squeeze())

In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#for X, y in test_dataloader:
#    print("Shape of X [N, C, H, W]: ", X.shape)
#    print("Shape of y: ", y.shape, y.dtype)
#    break

In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(561, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6)
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=561, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=6, bias=True)
  )
)


In [10]:
model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=561, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=6, bias=True)
  )
)

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        #print("pred:",pred.shape, pred.dtype," y:" ,y.shape,y.dtype)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 1200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.775056  [    0/  736]
Test Error: 
 Accuracy: 31.9%, Avg loss: 1.770597 

Epoch 2
-------------------------------
loss: 1.768396  [    0/  736]
Test Error: 
 Accuracy: 31.9%, Avg loss: 1.764070 

Epoch 3
-------------------------------
loss: 1.761971  [    0/  736]
Test Error: 
 Accuracy: 29.8%, Avg loss: 1.757759 

Epoch 4
-------------------------------
loss: 1.755720  [    0/  736]
Test Error: 
 Accuracy: 28.8%, Avg loss: 1.751631 

Epoch 5
-------------------------------
loss: 1.749627  [    0/  736]
Test Error: 
 Accuracy: 28.8%, Avg loss: 1.745680 

Epoch 6
-------------------------------
loss: 1.743690  [    0/  736]
Test Error: 
 Accuracy: 29.2%, Avg loss: 1.739902 

Epoch 7
-------------------------------
loss: 1.737901  [    0/  736]
Test Error: 
 Accuracy: 29.2%, Avg loss: 1.734287 

Epoch 8
-------------------------------
loss: 1.732246  [    0/  736]
Test Error: 
 Accuracy: 29.8%, Avg loss: 1.728807 

Epoch 9
----------------

In [21]:
torch.save(model.state_dict(), "activityModel.pth")

In [ ]:
model.load_state_dict(torch.load("activityModel.pth"))

In [ ]:
model.eval()

In [ ]:
# Let's do some predictions

with torch.no_grad():
    for X,y in test_dataloader:
        pred = model(X)
        print("Probabilities:",pred)
        print("Predicted class:",torch.argmax(pred, dim=1))
        print("match:",torch.argmax(pred, dim=1)==y)

In [ ]:
!pwd